<a href="https://colab.research.google.com/github/douglasepol/getdata-Redesocialcidades/blob/main/Redesocialcidades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lista de Todos os Indicadores


In [1]:
#!pip install python-dotenv

In [5]:
from dotenv import load_dotenv
import os
import requests
import psycopg2
from psycopg2.extras import execute_batch

def get_api_data(url):
    headers = {
        'Accept': 'application/json',
        'User-Agent': 'Mozilla/5.0'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print("Dados recebidos com sucesso!")
        return response.json()
    else:
        print("Erro na solicitação:", response.status_code)
        return None

def insert_data_to_db(db_params, query, data):
    try:
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                execute_batch(cur, query, data)
                print("Dados inseridos com sucesso.")
    except psycopg2.DatabaseError as e:
        print("Erro ao inserir dados:", e)

def main():
    load_dotenv()
    db_params = {
        'dbname': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD'),
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT')
    }

    # Defina a variável endpoint
    endpoint = 'variables'  # 'cities' ou 'indicators' ou 'variables'

    if endpoint == 'cities':
        url = 'https://www.redesocialdecidades.org.br/cities'
        insert_query = """
            INSERT INTO cidades (
                  longitude, name_uri, name, uf, pais, summary, latitude
              ) VALUES (
                  %(longitude)s, %(name_uri)s, %(name)s, %(uf)s, %(pais)s, %(summary)s, %(latitude)s
              );
        """
        table_key = 'cities'  # Chave esperada no JSON

    elif endpoint == 'indicators':
        url = 'https://www.redesocialdecidades.org.br/download-indicators?city_id=1'
        insert_query = """
            INSERT INTO dados_cidade (
                state_name, city_name, axis_name, formula, formula_human, institute_id, period, value
            ) VALUES (
                %(state_name)s, %(city_name)s, %(axis_name)s, %(formula)s, %(formula_human)s, %(institute_id)s,
                %(period)s, %(value)s
            );
        """
        table_key = 'data'  # Chave esperada no JSON

    elif endpoint == 'variables':
        url = 'https://www.redesocialdecidades.org.br/download-variables?city_id=1,398'
        insert_query = """
            INSERT INTO demographic_data (
                source, measurement_unit_name, valid_from, cognomen, city_name,
                variable_id, city_id, is_basic, institute_id, period, exp_source,
                value, name, observations, updated_at, type, user_id
            ) VALUES (
                %(source)s, %(measurement_unit_name)s, %(valid_from)s, %(cognomen)s, %(city_name)s,
                %(variable_id)s, %(city_id)s, %(is_basic)s, %(institute_id)s, %(period)s, %(exp_source)s,
                %(value)s, %(name)s, %(observations)s, %(updated_at)s, %(type)s, %(user_id)s
            );
        """
        table_key = 'data'  # Chave esperada no JSON

    data = get_api_data(url)

    if data and table_key in data:
        insert_data_to_db(db_params, insert_query, data[table_key])

if __name__ == "__main__":
    main()


Dados recebidos com sucesso!
Dados inseridos com sucesso.
